In [ ]:
# import os
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


In [ ]:
print(tf.__version__)

## Loading the digits and the objects from the data prep

In [ ]:
class DigitObject:
    def __init__(self, digit_image, cell_number, width, length, x_min, x_max, y_min, y_max, prediction=None):
        self.digit_image = digit_image
        self.cell_number = cell_number
        self.width = width
        self.length = length
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
        self.prediction = prediction

    def to_dict(self):
        return {
            'digit_image': self.digit_image,
            'cell_number': self.cell_number,
            'width': self.width,
            'length': self.length,
            'x_min': self.x_min,
            'x_max': self.x_max,
            'y_min': self.y_min,
            'y_max': self.y_max,
            'prediction': self.prediction
        }


In [ ]:
# Load the metadata JSON file
with open('../data-preparation/Data/It3/all_digit_objects_metadata.json', 'r') as json_file:
    loaded_metadata = json.load(json_file)

# Reconstruct the all_digit_objects dictionary with DigitObject instances
# all_digit_objects_loaded = {}

# for image_id, columns in loaded_metadata.items():
#     all_digit_objects_loaded[image_id] = {}

#     for col_name, digit_list in columns.items():
#         all_digit_objects_loaded[image_id][col_name] = []

#         for metadata in digit_list:
#             # Load the image from the saved path
#             digit_image = cv2.imread(metadata['image_path'], cv2.IMREAD_GRAYSCALE)

#             # Recreate the DigitObject
#             digit_obj = DigitObject(
#                 cell_number=metadata['cell_number'],
#                 width=metadata['width'],
#                 length=metadata['length'],
#                 x_min=metadata['x_min'],
#                 x_max=metadata['x_max'],
#                 y_min=metadata['y_min'],
#                 y_max=metadata['y_max'],
#                 prediction=metadata['prediction'],
#                 digit_image=digit_image
#             )

#             all_digit_objects_loaded[image_id][col_name].append(digit_obj)

# print("Dictionary and images loaded successfully.")


### checking that the dictionary was loaded propearly

In [ ]:
# for image_id, columns in  all_digit_objects_loaded.items():
#     print(f"Image ID: {image_id}")
#     for column_name, digit_list in columns.items():
#         print(f"  Column: {column_name}")
#         for digit_obj in digit_list:
#             # Print each attribute of the DigitObject instance
#             print(f"    Cell Number: {digit_obj.cell_number}")
#             print(f"    Width: {digit_obj.width}")
#             print(f"    Length: {digit_obj.length}")
#             print(f"    X Min: {digit_obj.x_min}")
#             print(f"    X Max: {digit_obj.x_max}")
#             print(f"    Y Min: {digit_obj.y_min}")
#             print(f"    Y Max: {digit_obj.y_max}")
#             print(f"    Prediction: {digit_obj.prediction}")
# 
#             # If you want to display the digit image
#             plt.figure(figsize=(2, 2))
#             plt.imshow(digit_obj.digit_image, cmap='gray')  # Assuming you saved the cropped image in `digit_obj.image`
#             plt.title(f"Digit in {digit_obj.cell_number}")
#             plt.show()


## Running CNN Model to train it 

In [ ]:
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
Y_train = train["label"]
X_train = train.drop(labels = ["label"], axis = 1)

In [ ]:
X_train = X_train / 255.0
test = test / 255.0

In [ ]:
X_train.values.shape

In [ ]:
Y_train = keras.utils.to_categorical(Y_train, num_classes=10)

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.3, random_state=42)

In [ ]:
# import scipy
# from scipy import ndimage
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# 
# datagen = ImageDataGenerator(
#     rotation_range=10,
#     zoom_range=0.1,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     horizontal_flip=False,
#     vertical_flip=False)

In [ ]:
# Assignment 2 - Reshape the data
X_train = X_train.values.reshape(-1, 28, 28, 1)
X_val = X_val.values.reshape(-1, 28, 28, 1)

print(X_train.shape) # Result: (33600, 28, 28, 1)
print(X_val.shape) # Result: (8400, 28, 28, 1)

In [ ]:
# Assignment 2 - Create a neural network with dropout layers
model = keras.models.Sequential()

model.add(keras.layers.Flatten(input_shape=(28, 28, 1)))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9),loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
# # Assignment 1 - Train the model
model.fit(
X_train,
Y_train,
batch_size=32,
epochs=15,
validation_data=(X_val, Y_val)
)

In [ ]:
# # Assignment 2 - Train the model with ImageDataGenerator
# import scipy
# import scipy.ndimage

# model.fit(
#     datagen.flow(X_train, Y_train, batch_size=24),
#     epochs=15,
#     validation_data=(X_val, Y_val)
# )

In [ ]:
# Assignment 2
predictions = model.predict(test.values.reshape(-1, 28, 28, 1))

print(predictions[:5])

results = np.argmax(predictions, axis = 1)

print(results[:5])

## Running the model to predict the images digits

takes 15 minutes to run for the 60 images digits (over 9000 digits))

In [ ]:
import os
import json
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the metadata JSON file
with open('../data-preparation/Data/It3/all_digit_objects_metadata.json', 'r') as json_file:
    loaded_metadata = json.load(json_file)

# Function to preprocess the image and predict the digit
def predict_digit(image_path):
    # Normalize the path and replace backslashes with forward slashes
    normalized_image_path = os.path.normpath(f"../data-preparation/{image_path}").replace("\\", "/")

    # Check if the file exists before attempting to load it
    if not os.path.exists(normalized_image_path):
        print(f"Warning: Image path {normalized_image_path} does not exist.")
        return None

    # Load the image in grayscale
    digit_image = cv2.imread(normalized_image_path, cv2.IMREAD_GRAYSCALE)
    
    if digit_image is None:
        print(f"Warning: Image at {normalized_image_path} could not be loaded.")
        return None

    # Preprocess the image
    digit_image = digit_image / 255.0  # Normalize pixel values
    digit_image = cv2.resize(digit_image, (28, 28))  # Ensure image size is 28x28
    digit_image = np.expand_dims(digit_image, axis=-1)  # Add channel dimension (28, 28, 1)
    digit_image = np.expand_dims(digit_image, axis=0)   # Add batch dimension (1, 28, 28, 1)

    # Predict the digit
    prediction = model.predict(digit_image)
    predicted_digit = int(np.argmax(prediction))  # Convert int64 to int

    return predicted_digit

# Update the digit objects with predictions
for image_id, columns in loaded_metadata.items():
    for col_name, digit_list in columns.items():
        for digit_obj in digit_list:
            # Update the image path with the correct folder path
            image_path = digit_obj['image_path']
            
            # Predict the digit using the model
            predicted_digit = predict_digit(image_path)

            # Update the prediction field in the metadata
            if predicted_digit is not None:
                digit_obj['prediction'] = predicted_digit

# Save the updated metadata to a JSON file
with open('all_digit_objects_with_predictions.json', 'w') as json_file:
    json.dump(loaded_metadata, json_file)

print("Predictions added and saved successfully.")


## Organizing the JSON File to be readable

In [ ]:
import json

# Load the JSON file
with open('all_digit_objects_with_predictions.json', 'r') as json_file:
    loaded_metadata = json.load(json_file)

# Save the JSON file with organized formatting
with open('all_digit_objects_with_predictions_organized.json', 'w') as json_file:
    json.dump(loaded_metadata, json_file, indent=4, sort_keys=True)

print("The JSON file has been organized and saved successfully.")


## Printing the predictions against the saved digits for wasy comparison

In [ ]:
import os
import cv2

with open('all_digit_objects_with_predictions_organized.json', 'r') as json_file:
    loaded_metadata = json.load(json_file)

# Reconstruct the all_digit_objects dictionary with DigitObject instances
all_digit_objects_loaded = {}

for image_id, columns in loaded_metadata.items():
    all_digit_objects_loaded[image_id] = {}

    for col_name, digit_list in columns.items():
        all_digit_objects_loaded[image_id][col_name] = []

        for metadata in digit_list:
            # Update the image path with the correct folder path
            image_path = metadata['image_path']
            normalized_image_path = os.path.normpath(f"../data-preparation/{image_path}").replace("\\", "/")

            # Load the image from the normalized path
            digit_image = cv2.imread(normalized_image_path, cv2.IMREAD_GRAYSCALE)

            if digit_image is None:
                print(f"Warning: Image at {normalized_image_path} could not be loaded.")
                continue

            # Recreate the DigitObject
            digit_obj = DigitObject(
                cell_number=metadata['cell_number'],
                width=metadata['width'],
                length=metadata['length'],
                x_min=metadata['x_min'],
                x_max=metadata['x_max'],
                y_min=metadata['y_min'],
                y_max=metadata['y_max'],
                prediction=metadata['prediction'],
                digit_image=digit_image
            )

            all_digit_objects_loaded[image_id][col_name].append(digit_obj)

print("Dictionary and images loaded successfully.")


In [ ]:
import random
import matplotlib.pyplot as plt

# Flatten the list of digit objects to sample from all of them
all_digits = []

for image_id, columns in all_digit_objects_loaded.items():
    for column_name, digit_list in columns.items():
        for digit_obj in digit_list:
            all_digits.append((image_id, column_name, digit_obj))

# Select 30 random digit objects to display
random_digits = random.sample(all_digits, 30)

# Display the selected random digit objects
for image_id, column_name, digit_obj in random_digits:
    print(f"Image ID: {image_id}")
    print(f"  Column: {column_name}")
    print(f"    Cell Number: {digit_obj.cell_number}")
    print(f"    Width: {digit_obj.width}")
    print(f"    Length: {digit_obj.length}")
    print(f"    X Min: {digit_obj.x_min}")
    print(f"    X Max: {digit_obj.x_max}")
    print(f"    Y Min: {digit_obj.y_min}")
    print(f"    Y Max: {digit_obj.y_max}")
    print(f"    Prediction: {digit_obj.prediction}")

    # Display the digit image
    plt.figure(figsize=(2, 2))
    plt.imshow(digit_obj.digit_image, cmap='gray')
    plt.title(f"Digit in {digit_obj.cell_number}")
    plt.show()
